<a href="https://colab.research.google.com/github/shreyangshu12/Next-word-prediction-using-LSTM/blob/main/Next_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import heapq
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation
from keras.layers import *
import pickle
from tensorflow.keras.optimizers import RMSprop
import re
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
#open text file in read mode
path = "1661-0.txt"
#read whole file to a string
data = open(path).read().lower()
data = re.sub(r'[^\w\s]', '', data)
data = re.sub(" \d+", " ", data)
 
#print(data)

In [ ]:
tokenizer = RegexpTokenizer(r'\w+|$[0-9]+|\S+')
words = tokenizer.tokenize(data)
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
len(words)

107435

In [ ]:
LENGTH_WORD = 5
next_words = []
prev_words = []
for j in range(len(words) - LENGTH_WORD):
  prev_words.append(words[j:j + LENGTH_WORD])
  next_words.append(words[j + LENGTH_WORD])
print(prev_words[0])
print(next_words[0])

['project', 'gutenbergs', 'the', 'adventures', 'of']
sherlock


In [ ]:
arr = np.zeros((len(prev_words), LENGTH_WORD), dtype=int)
for i in range(len(prev_words)):
  for j in range(5):
    arr[i][j] = unique_word_index[prev_words[i][j]]

In [ ]:
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
   for j, each_word in enumerate(each_words):
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(arr, Y, test_size=0.05, random_state=42)

In [ ]:
inputs=Input(shape=(5,))

embedding=Embedding(input_dim=len(unique_words),
             output_dim=300,
             input_length=5,
             embeddings_regularizer=keras.regularizers.l2(.001))(inputs)

bilstm_0 = Bidirectional(CuDNNLSTM(units=128, return_sequences=True))(embedding)

bilstm_1 = Bidirectional(CuDNNLSTM(units=128, return_sequences=False))(bilstm_0)


output=Dense(len(unique_words),activation='softmax')(bilstm_1) 

model=keras.Model(inputs,output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 embedding (Embedding)       (None, 5, 300)            2599200   
                                                                 
 bidirectional (Bidirectiona  (None, 5, 256)           440320    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              395264    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 8664)              2226648   
                                                                 
Total params: 5,661,432
Trainable params: 5,661,432
Non-train

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

checkpoint_filepath = '/content/drive/MyDrive/Next_word_prediction_weights.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                               mode='auto', 
                                               patience=3, 
                                               verbose=1)
history = model.fit(X_train, y_train, 
                    validation_split=0.05, 
                    batch_size=128, 
                    epochs=50, 
                    shuffle=True,
                    callbacks=[early_stopping,model_checkpoint_callback])

Epoch 1/50
758/758 [==============================] - 17s 12ms/step - loss: 6.6273 - accuracy: 0.0617 - val_loss: 6.3238 - val_accuracy: 0.0766
Epoch 2/50
758/758 [==============================] - 8s 11ms/step - loss: 5.9992 - accuracy: 0.0928 - val_loss: 6.0472 - val_accuracy: 0.1068
Epoch 3/50
758/758 [==============================] - 8s 11ms/step - loss: 5.7340 - accuracy: 0.1134 - val_loss: 5.9883 - val_accuracy: 0.1219
Epoch 4/50
758/758 [==============================] - 9s 11ms/step - loss: 5.5580 - accuracy: 0.1232 - val_loss: 5.9391 - val_accuracy: 0.1274
Epoch 5/50
758/758 [==============================] - 8s 11ms/step - loss: 5.4114 - accuracy: 0.1297 - val_loss: 5.9178 - val_accuracy: 0.1340
Epoch 6/50
758/758 [==============================] - 9s 11ms/step - loss: 5.2800 - accuracy: 0.1354 - val_loss: 5.9352 - val_accuracy: 0.1368
Epoch 7/50
758/758 [==============================] - 8s 11ms/step - loss: 5.1572 - accuracy: 0.1403 - val_loss: 5.9223 - val_accuracy: 0.140

In [ ]:
model.evaluate(X_test, y_test)

168/168 [==============================] - 1s 6ms/step - loss: 6.1221 - accuracy: 0.1340


[6.122071743011475, 0.13402830064296722]

In [ ]:
def prepare_input(text, SEQUENCE_LENGTH=5):
  x = np.zeros((1, SEQUENCE_LENGTH))
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(" \d+", " ", text)    
  text = text.split()
  for i in range(5):
    x[0][i] = unique_word_index[text[i]]
  return x

def predicted(text):
  x = prepare_input(text)
  list_of_keys = list(unique_word_index.keys())
  predicted = model.predict(x)
  print(list_of_keys[np.argmax(predicted)])

In [ ]:
predicted('there we are sitting alone')

in
